In [1]:
library(plyr)
library(ggplot2)

In [2]:
#genomeScores <- read.csv("C:/Users/matt/Downloads/ml-20m/ml-20m/genome-scores.csv",stringsAsFactors = FALSE)
genomeTags <- read.csv("C:/Users/matt/Downloads/ml-20m/ml-20m/genome-tags.csv",stringsAsFactors = FALSE)

#ratings <- read.csv("C:/Users/matt/Downloads/ml-20m/ml-20m/ratings.csv",stringsAsFactors = FALSE)
#randomRatings <- ratings[sample(nrow(ratings)),]
#trainingRatings <- randomRatings [1:16000000,]
#testRatings <- randomRatings [16000001:nrow(randomRatings),]
#write.table(trainingRatings,"C:/temp/trainingRatings.txt")
#write.table(testRatings,"C:/temp/testRatings.txt")

In [3]:
movies <- read.csv("C:/Users/matt/Downloads/ml-20m/ml-20m/movies.csv",stringsAsFactors = FALSE)

In [4]:
trainRating <- read.table("C:/temp/trainingRatings.txt",nrow=900000)
#testRating <- read.table("C:/temp/testRatings.txt")

In [5]:

#trimmedGenomeScores <- subset(genomeScores, tagId %in% sample(1128,100))
#write.table(trimmedGenomeScores, "C:/temp/trimmedGenomeScores.txt")

trimmedGenomeScores <- read.table("C:/temp/trimmedGenomeScores.txt")


In [6]:
TagAffinity2 <- function(thisUserId)
{

	usersReviews = subset(trainRating, userId == thisUserId)
	usersTagAffinity = data.frame(tagId = trimmedGenomeScores$tagId[1:100], constant = rep(0,100),
					linearCoeff = rep(0,100), error=rep(0,100))
	movietags2 = subset(trimmedGenomeScores, movieId %in% usersReviews$movieId)
	movietags2=movietags2[order(movietags2$movieId, movietags2$tagId),]
	usableReviews = subset(usersReviews, movieId %in% unique(movietags2$movieId))
	usableReviews = usableReviews[order(usableReviews$movieId),]

	for (j in 1:100)
	{
		tags= movietags2[(0:((nrow(movietags2)-1)/100))*100 + j,]
		fit = summary(lm(formula=usableReviews$rating~tags$relevance, weight=tags$relevance))
		usersTagAffinity$constant[j]=fit$coefficients[1,1]
		usersTagAffinity$linearCoeff[j]=fit$coefficients[2,1]
		usersTagAffinity$error[j] = fit$coefficients[2,4]
	}
return(usersTagAffinity)
}


In [7]:

powerUserAffinities <- vector(mode="list", length=length(unique(powerUserTestRatings$userId)))
names(powerUserAffinities) = unique(powerUserTestRatings$userId)


ERROR: Error in unique(powerUserTestRatings$userId): object 'powerUserTestRatings' not found


In [ ]:
for (i in unique(powerUserTestRatings$userId))
{
	powerUserAffinities[[as.character(i)]] = TagAffinity2(i)
	#print(count)
	count = count + 1
}

In [ ]:

UserFits <- function(thisUserId)
{
thisUsersTests = subset(testRating,userId == thisUserId)
affinityStrengths = NULL
count = 1
basetags = subset(genomeScores, tagId %in% trimmedGenomeScores$tagId & movieId %in% thisUsersTests$movieId)
affinities = powerUserAffinities[[as.character(thisUserId)]]
for (filmId in thisUsersTests$movieId)
{
	tags = subset(basetags , movieId == filmId)
	weights = (tags$relevance^3)/sum(tags$relevance^3)
	affinityStrengths[count] = sum(weights*(affinities$constant +affinities$linearCoeff*tags$relevance))
	count = count + 1
	#print(count)
}
toReturn=data.frame(movieId = thisUsersTests$movieId, predictions = affinityStrengths)
return(subset(toReturn,))
}


In [ ]:

UserFilmPredictions <- vector(mode="list", length=length(unique(powerUserTestRatings$userId)))
names(powerUserAffinities) = unique(powerUserTestRatings$userId)
for (user in unique(powerUserTestRatings$userId))
{
	predictions=UserFits(user)
	predictions=predictions[order(predictions$movieId),]
	ratings=subset(testRating, user==userId)
	ratings=ratings[order(ratings$movieId),]
	predictions$actualRating=ratings$rating
	predictions$differenceFromMean=abs(predictions$actualRating - mean(testRating$rating))
	predictions$differenceFromPrediction=abs(predictions$actualRating - predictions$prediction)
	predictions$userId = user
	UserFilmPredictions[[as.character(user)]]=predictions
	print(user)
}
bigList=do.call("rbind",UserFilmPredictions)
bigList2=subset(bigList, predictions != 0)
